In [1]:
import tensorflow as tf
from src.generator import DataGen
import numpy as np
import faiss
from src.label2metric import ind2label
import json

In [2]:
dg = DataGen(input_shape=(224,224,3),data_path="src\data",batch_size=1)


initialising train generator...
Generator fully initialised, 10850 samples available


In [3]:

emb = np.load("src/data/datasets/emb_ind/hackING23/embeddings.npy")
idx = np.load("src/data/datasets/emb_ind/hackING23/indices.npy")[:-1] #todo to be change
class_idxs = np.load("src/notebooks/class_idx.npy") 


In [4]:
print(emb.shape)
print(idx.shape)
print(idx[0])
print(idx[1])
labels = dg._make_index()
labels[0:5]
#print(class_idxs)


(10848, 768)
(10848,)
0
1


,fname,label
0,00133f89-017d-44f5-a549-e11cf5b5221a.tiff,advertisement
1,00a3e900-6616-46fe-a528-a5ac51952562.tiff,advertisement
2,0116aef3-5834-4a14-8dc0-e7eb3d79f5de.tiff,advertisement
3,024b107d-cd7f-4f81-a28c-50a9df08d586.tiff,advertisement
4,0296a9e1-5bec-456f-9e9c-fc8849203056.tiff,advertisement


In [5]:
def unison_shuffled_copies(a, b, c,d):
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p],d[p]

In [6]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(labels.label)
print(le.classes_)
labels_dupa = le.transform(labels.label)
labels["label_id"] = labels_dupa

['advertisement' 'budget' 'email' 'file_folder' 'form' 'handwritten'
 'invoice' 'letter' 'memo' 'news_article' 'pit37_v1'
 'pozwolenie_uzytkowanie_obiektu_budowlanego' 'presentation'
 'questionnaire' 'resume' 'scientific_publication' 'scientific_report'
 'specification' 'umowa_na_odleglosc_odstapienie' 'umowa_o_dzielo'
 'umowa_sprzedazy_samochodu']


In [7]:


emb_suffled, idx_suffled, filenavmes_sh,labels_shafle = unison_shuffled_copies(emb,idx, labels.fname,labels.label_id)

label_dict = dict(zip(filenavmes_sh,labels_shafle))

print(label_dict["0296a9e1-5bec-456f-9e9c-fc8849203056.tiff"])


0


In [8]:

# ranges and classes inds
ranges = np.load('src/notebooks/class_idx.npy')
print(ranges.shape)
# classes names
f  =  open("src/data/hackathon/label2id_final.json")
class_names = json.load(f)

(21, 3)


In [9]:
start = 0
d = 768
k = 1
for i in range(1,11):
    counter = 0
    stop = int((10848*i/10))
    
    indexes = np.zeros((10848),dtype=np.int32)
    indexes[start:stop] = 1
    query_emb = emb_suffled[np.where(indexes==1)]
    query_idx = idx_suffled[np.where(indexes==1)]
    query_len = len(query_idx)

    #what left
    #db = emb_suffled[np.where(indexes==0)[0]]
    db_idx = idx_suffled[np.where(indexes==0)[0]]

    # for j in idx_suffled[np.where(indexes==0)[0]]:
    #     idx_for_db.append(np.argwhere(idx ==j))
    db = np.copy(emb)
    db[query_idx] = 9
    index = faiss.IndexFlatL2(d)    # build the index
    index.add(db)                   # add vectors to the index
                                    # we want to see 1 nearest neighbors

    _, I = index.search(query_emb, k)               # sanity check
    files_names = list(labels.fname[query_idx])
    match_labels = ind2label(I,files_names,ranges)
    counter_class_T = 0
    counter_class_F = 0
    #class_id = 7
    for key,value in match_labels.items():
        
        if label_dict[key] == value:# and label_dict[key]==class_id:
            counter_class_T+=1
        elif label_dict[key] != value:# and label_dict[key]==class_id:
            counter_class_F+=1
            #print(label_dict[key],key,value)
    #print(counter_class_T/(counter_class_T+counter_class_F))
        
    

    # labels.label[]

    

    start = stop
    print(counter_class_T/(counter_class_F+counter_class_T))

0.5922509225092251
0.6082949308755761
0.5668202764976958
0.5870967741935483
0.5907834101382489
0.6097785977859779
0.6055299539170507
0.5953917050691244
0.552073732718894
0.5658986175115207


In [93]:
d = 768
index = faiss.IndexFlatL2(d)   # build the index
index.add(emb)                  # add vectors to the index
print(index.is_trained)

k = 20                          # we want to see 4 nearest neighbors

True
